# FuzzyCocoPython quickstart

This notebook gives a compact tour of the scikit-learn style API exposed by `fuzzycocopython`.
We train a classifier, a regressor, inspect generated fuzzy rules, and show how to persist a model.

We will:
- fit a `FuzzyCocoClassifier` on the Iris dataset and review its fuzzy knowledge base
- fit a `FuzzyCocoRegressor` on the Diabetes dataset and inspect predictions
- compute rule activation statistics
- save and reload a trained estimator

In [1]:
import numpy as np
import pandas as pd
from pathlib import Path

from sklearn.datasets import load_iris, load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.metrics import root_mean_squared_error

from fuzzycocopython import FuzzyCocoClassifier, FuzzyCocoRegressor

RANDOM_STATE = 0

## Classification example

In [2]:
iris = load_iris(as_frame=True)
X_iris = iris.data
y_iris = iris.target

X_train, X_test, y_train, y_test = train_test_split(
    X_iris,
    y_iris,
    test_size=0.3,
    stratify=y_iris,
    random_state=RANDOM_STATE,
)

X_train.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
144,6.7,3.3,5.7,2.5
117,7.7,3.8,6.7,2.2
115,6.4,3.2,5.3,2.3
22,4.6,3.6,1.0,0.2
28,5.2,3.4,1.4,0.2


In [3]:
clf = FuzzyCocoClassifier(random_state=RANDOM_STATE)
clf.fit(X_train, y_train)

,params,None
,random_state,0
,params_overrides,None


In [4]:
iris_predictions = pd.DataFrame({
    "y_true": y_test.reset_index(drop=True),
    "y_pred": clf.predict(X_test),
})

print(f"Accuracy: {clf.score(X_test, y_test):.3f}")
iris_predictions.head()

Accuracy: 0.667


,y_true,y_pred
0,2,1
1,2,1
2,0,0
3,0,0
4,1,1


### Inspect the fuzzy system

In [5]:
# Each row corresponds to a fuzzy rule described by the engine
clf.rules_df_.head()

,rule,role,io,var,label,position,orig_set
0,default,default,output,target,High,0.956863,target.2
1,rule1,antecedent,input,petal length (cm),Low,2.550196,petal length (cm).1
2,rule1,antecedent,input,petal width (cm),Low,0.871765,petal width (cm).1
3,rule1,antecedent,input,sepal width (cm),High,2.216471,sepal width (cm).2
4,rule1,consequent,output,target,Low,0.423529,target.1


In [6]:
rule_stats = clf.rules_stat_activations(X_test)
rule_stats.head()


,mean,std,min,max,usage_rate,usage_rate_pct,importance_pct,impact_pct
"IF (sepal width (cm) is High AND_min petal length (cm) is Low AND_min petal width (cm) is Low), THEN (target is Low)",0.508331,0.379589,0.0,1.0,0.911111,91.111111,91.111111,83.012346


In [7]:
# simply look at the learned rules
clf.rules_, clf.default_rules_

([IF (sepal width (cm) is High AND_min petal length (cm) is Low AND_min petal width (cm) is Low), THEN (target is Low)],
 [ELSE (target is High)])

## Regression example

In [8]:
diabetes = load_diabetes(as_frame=True)
X_reg = diabetes.data
y_reg = diabetes.target

X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(
    X_reg,
    y_reg,
    test_size=0.25,
    random_state=RANDOM_STATE,
)

X_train_reg.head()

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6
20,-0.049105,-0.044642,-0.056863,-0.043542,-0.045599,-0.043276,0.000779,-0.039493,-0.011897,0.015491
353,-0.052738,-0.044642,-0.055785,-0.036656,0.089244,-0.003193,0.008142,0.034309,0.132376,0.003064
281,-0.092695,0.050680,-0.090275,-0.057313,-0.024960,-0.030437,-0.006584,-0.002592,0.024055,0.003064
14,0.045341,-0.044642,-0.025607,-0.012556,0.017694,-0.000061,0.081775,-0.039493,-0.031988,-0.075636
300,0.016281,-0.044642,0.073552,-0.041235,-0.004321,-0.013527,-0.013948,-0.001116,0.042897,0.044485


In [9]:
reg = FuzzyCocoRegressor(random_state=RANDOM_STATE)
reg.fit(X_train_reg, y_train_reg)

,params,None
,random_state,0
,params_overrides,None


In [10]:
reg_predictions = reg.predict(X_test_reg)
print(f"R^2 score: {reg.score(X_test_reg, y_test_reg):.3f}")
print(f"RMSE: {root_mean_squared_error(y_test_reg, reg_predictions):.2f}")

pd.DataFrame({
    "y_true": y_test_reg.reset_index(drop=True),
    "y_pred": reg_predictions,
}).head()

R^2 score: -0.256
RMSE: 78.96


,y_true,y_pred
0,321.0,251.749522
1,215.0,106.823529
2,127.0,181.371171
3,64.0,106.823529
4,175.0,106.823529


In [11]:
reg.rules_df_.head()

,rule,role,io,var,label,position,orig_set
0,default,default,output,target,Low,106.823529,target.1
1,rule1,antecedent,input,age,Low,0.010725,age.1
2,rule1,antecedent,input,bmi,High,0.045766,bmi.2
3,rule1,antecedent,input,bp,Low,0.064942,bp.1
4,rule1,antecedent,input,sex,High,0.006944,sex.2


## Persisting a trained model

In [12]:
artifact = Path("fuzzycoco_classifier.pkl")
clf.save(artifact)
loaded_clf = FuzzyCocoClassifier.load(artifact)

print("Predictions from the reloaded model:")
print(loaded_clf.predict(X_test.iloc[:3]))

artifact.unlink(missing_ok=True)

Predictions from the reloaded model:
[1 1 0]
